<a href="https://colab.research.google.com/github/johansbustamante-gif/Proyecto-Inteligencia-Artificial/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Desinstalar y reinstalar lightgbm y dask para resolver posibles conflictos de dependencia con cupy
# Esto es común en entornos como Colab para asegurar una instalación limpia.
!pip uninstall -y lightgbm dask
!pip install lightgbm dask[array]

print("✅ `lightgbm` y `dask` reinstalados. Por favor, reinicia el entorno de ejecución (Runtime -> Restart runtime) y luego ejecuta tus celdas nuevamente.")


Found existing installation: lightgbm 4.6.0
Uninstalling lightgbm-4.6.0:
  Successfully uninstalled lightgbm-4.6.0
Found existing installation: dask 2025.9.1
Uninstalling dask-2025.9.1:
  Successfully uninstalled dask-2025.9.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rapids-dask-dependency 25.10.0 requires dask==2025.9.1, but you have dask 2025.11.0 which is incompatible.
distributed 2025.9.1 requires dask==2025.9.1, but you have dask 2025.11.0 which is incompatible.
✅ `lightgbm` y `dask` reinstalados. Por favor, reinicia el entorno de ejecución (Runtime -> Restart runtime) y luego ejecuta tus celdas nuevamente.


In [5]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
import os

# 1. CARGA DE DATOS (Asegúrate de ajustar la ruta si usas Drive o local)
# Simulamos carga para el ejemplo, reemplaza con tus rutas

from google.colab import drive
print("Montando Google Drive...")
drive.mount('/content/drive', force_remount=True)

DRIVE_PATH = '/content/drive/MyDrive/DataProyectoIA'
TRAIN_PATH = os.path.join(DRIVE_PATH, 'train.csv')
TEST_PATH = os.path.join(DRIVE_PATH, 'test.csv')

if not os.path.exists(DRIVE_PATH):
    print("ERROR: No existe la carpeta esperada:", DRIVE_PATH)
    print("Contenido de /content/drive/MyDrive:")
    try:
        print(os.listdir('/content/drive/MyDrive'))
    except Exception:
        pass
    raise FileNotFoundError("Ajusta DRIVE_PATH")

if not os.path.exists(TRAIN_PATH):
    print("ERROR: No se encontró:", TRAIN_PATH)
    print("Contenido de", DRIVE_PATH, ":", os.listdir(DRIVE_PATH))
    raise FileNotFoundError("Coloca train_limpio.csv en la carpeta o corrige la ruta")

# Cargar los datasets
df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)

# 2. LIMPIEZA Y MANEJO DE NULOS
def basic_cleaning(df):
    # Separar numéricas y categóricas
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

    # Imputar Categóricas con 'Missing' (Mejor que moda para árboles de decisión)
    for col in cat_cols:
        df[col] = df[col].fillna('Missing')

    # Imputar Numéricas con Mediana
    for col in num_cols:
        if col != 'ID': # No imputar ID
            df[col] = df[col].fillna(df[col].median())
    return df, cat_cols

print("Limpiando datos...")
df_train, cat_features = basic_cleaning(df_train)
df_test, _ = basic_cleaning(df_test)


Montando Google Drive...
Mounted at /content/drive
Limpiando datos...


In [6]:
# 3. PREPARACIÓN PARA LIGHTGBM (SIN ONE-HOT ENCODING MASIVO)
# LightGBM maneja categorías nativamente si se pasan como tipo 'category' de pandas
# Esto ahorra RAM y mejora precisión en alta cardinalidad.

# Eliminamos ID y Target de las features
X = df_train.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1)
y = df_train['RENDIMIENTO_GLOBAL']
X_test = df_test.drop(['ID'], axis=1) # Test no tiene target

# Mapeo del Target a números
target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
y = y.map(target_map)

# Manejar NaNs en el target 'y'
# Si hay NaNs en y después del mapeo, debemos eliminarlos junto con las filas correspondientes en X.
nan_indices = y[y.isna()].index
if not nan_indices.empty:
    print(f"⚠️ Se encontraron {len(nan_indices)} NaNs en el target 'RENDIMIENTO_GLOBAL' después del mapeo. Se eliminarán estas filas de X y y.")
    X = X.drop(nan_indices)
    y = y.drop(nan_indices)


# Codificación de Features Categóricas a Números (Label Encoding)
# LightGBM prefiere enteros para categorías, no strings ni One-Hot gigante
# Primero, elimina 'RENDIMIENTO_GLOBAL' de cat_features si está presente, ya que es el target.
cat_features_for_encoding = [col for col in cat_features if col != 'RENDIMIENTO_GLOBAL']

for col in cat_features_for_encoding:
    if col in X.columns: # Verificar que no sea el target
        le = LabelEncoder()
        # Ajustamos con ambos sets para no tener errores de categorías nuevas
        combined_data = pd.concat([X[col], X_test[col]], axis=0).astype(str)
        le.fit(combined_data)
        X[col] = le.transform(X[col].astype(str))
        X_test[col] = le.transform(X_test[col].astype(str))

        # Convertir a tipo 'category' para que LightGBM sepa qué hacer
        X[col] = X[col].astype('category')
        X_test[col] = X_test[col].astype('category')

# Corrección de nombres de columnas para evitar el error de caracteres JSON
X.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X.columns]
X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

In [7]:
# 4. DIVISIÓN (SPLIT)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
# 5. ENTRENAMIENTO EFICIENTE
print("Entrenando LightGBM...")
model = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.1,
    num_leaves=31,
    random_state=42,
    n_jobs=-1, # Ahora sí es seguro usar -1 porque no hay 1000 columnas
    verbose=-1
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='multi_logloss'
    # callbacks=[early_stopping(10)] # Opcional si tienes versión reciente
)

Entrenando LightGBM...


LGBMClassifier(n_estimators=200, n_jobs=-1, random_state=42, verbose=-1)

In [9]:
# 6. EVALUACIÓN
val_preds = model.predict(X_val)
acc = accuracy_score(y_val, val_preds)
print(f"\n✅ Accuracy en Validación: {acc:.4f}")
print(classification_report(y_val, val_preds, target_names=target_map.keys()))


✅ Accuracy en Validación: 0.4390
              precision    recall  f1-score   support

        bajo       0.47      0.57      0.51     34597
  medio-bajo       0.33      0.28      0.31     34455
  medio-alto       0.33      0.28      0.30     34324
        alto       0.57      0.62      0.59     35124

    accuracy                           0.44    138500
   macro avg       0.42      0.44      0.43    138500
weighted avg       0.43      0.44      0.43    138500



In [10]:
# 7. SUBMISSION
print("Generando predicciones finales...")
test_preds = model.predict(X_test)

# Invertir mapeo
inv_map = {v: k for k, v in target_map.items()}
submission = pd.DataFrame({
    'ID': df_test['ID'],
    'RENDIMIENTO_GLOBAL': [inv_map[p] for p in test_preds]
})

submission.to_csv('submission_optimizado.csv', index=False)
print("Archivo guardado: submission_optimizado.csv")

Generando predicciones finales...
Archivo guardado: submission_optimizado.csv
